In report:<br>
- Talk about why none label is necessary with LLM (might not predict one of the given emotions)<br>
- How problem can be alleviated / solved (checking for synonyms, disincentivise model to pick none with very low accuracy etc)

In [1]:
#%pip uninstall torch torchvision torchaudio
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [1]:
!nvidia-smi

Tue Apr  9 16:56:41 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.52                 Driver Version: 551.52         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| 44%   31C    P8             13W /  125W |     815MiB /   6144MiB |     12%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%pip install -Uqqq pip --progress-bar off
%pip install -qqq torch --progress-bar off
%pip install -qqq transformers --progress-bar off
%pip install -qqq datasets --progress-bar off
%pip install -qqq peft==0.5.0 --progress-bar off
%pip install -qqq bitsandbytes==0.43.0 --progress-bar off
%pip install -qqq trl==0.7.11 --progress-bar off # trl - Training Reinforcement Learning (FOR THE TRAINER)
%pip install -qqq -U scikit-learn --progress-bar off
# %pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12

: 

In [2]:
%pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
  Using cached bitsandbytes-0.43.0-py3-none-win_amd64.whl.metadata (1.9 kB)
Using cached bitsandbytes-0.43.0-py3-none-win_amd64.whl (101.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [13]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# torch.__version__

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.17.2%2Bcu118-cp312-cp312-win_amd64.whl (4.9 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.2.2%2Bcu118-cp312-cp312-win_amd64.whl (3.9 MB)
  Using cached https://download.pytorch.org/whl/cu118/torch-2.2.2%2Bcu118-cp312-cp312-win_amd64.whl (2704.2 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1
    Uninstalling torch-2.2.1:
      Successfully uninstalled torch-2.2.1
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.


In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

# %pip install kaggle
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'

In [7]:
#%pip install bitsandbytes --prefer-binary --extra-index-url=https://jllllll.github.io/bitsandbytes-windows-webui

In [1]:
%pip install -qqq ipywidgets==8.1.2

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import re
from pprint import pprint
import numpy as np
import pandas as pd
#import bitsandbytes

# %load_ext cudf.pandas  # pandas operations now use the GPU!
import torch
from peft import LoraConfig, PeftModel
from huggingface_hub import notebook_login
from transformers import(
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from trl import SFTTrainer
from trl import setup_chat_format
from tqdm import tqdm
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

DEVICE =  "cuda: 0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Llama-2-7b-hf"
DEVICE

#LD_LIBRARY_PATH = 

'cuda: 0'

# **Dataset**

In [3]:
from datasets import load_dataset, Dataset, Value, ClassLabel, Features

my_dataset = load_dataset("./Emotion_Dataset", sep=",")
# Creating a ClassLabel Object
df = my_dataset["train"].to_pandas()
labels = ['sadness','joy','love','anger', 'fear', 'surprise']
ClassLabels = ClassLabel(num_classes=len(labels), names=labels)

# Mapping Labels to IDs
def map_label2id(example):
    example['label'] = ClassLabels.str2int(example['label'])
    return example

my_dataset= my_dataset.map(map_label2id, batched=True)

# Casting label column to ClassLabel Object
my_dataset = my_dataset.cast_column('label', ClassLabels)

my_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [4]:
dataset_df = my_dataset["train"].to_pandas()
dataset_df

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,1
15998,i feel like this was such a rude comment and i...,3


In [5]:
features = my_dataset["train"].features
features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [6]:
id2label = {idx:features["label"].int2str(idx) for idx in range(6)}
id2label

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [7]:
label2id = {v:k for k,v in id2label.items()}
label2id

{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

In [8]:
dataset_df["label"].value_counts(normalize=True).sort_index()

label
0    0.291625
1    0.335125
2    0.081500
3    0.134937
4    0.121063
5    0.035750
Name: proportion, dtype: float64

In [9]:
class_weights = (1 - (dataset_df["label"].value_counts().sort_index() / len(dataset_df))).values #assigning high weight to the rare classes and low weight to the common classes to balance it out as there is an imbalance in the dataset
class_weights


class_weights = torch.from_numpy(class_weights).float()
class_weights = class_weights.to("cuda")
class_weights

tensor([0.7084, 0.6649, 0.9185, 0.8651, 0.8789, 0.9643], device='cuda:0')

In [10]:
X_train = my_dataset["train"].to_pandas()
X_test = my_dataset["test"].to_pandas()
X_valid = my_dataset["validation"].to_pandas()

def generate_prompt(data_point):
    return f"""
            Analyze the emotion of the texts in the square brackets,
            determine if it is sadness, joy, love, anger, fear, surprise, and return the answer as
            the corresponding sentiment label "sadness","joy","love","anger", "fear", "surprise".

            [{data_point["text"]}] = {data_point["label"]}
            """.strip()

def generate_test_prompt(data_point):
    return f"""
            Analyze the emotion of the texts in the square brackets,
            determine if it is sadness, joy, love, anger, fear, surprise, and return the answer as
            the corresponding sentiment label 'sadness','joy','love','anger', 'fear', 'surprise'.

            [{data_point["text"]}] = """.strip()

X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1),
                       columns=["text"])
X_valid = pd.DataFrame(X_valid.apply(generate_prompt, axis=1),
                      columns=["text"])

y_true = X_test.label
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

train_data = Dataset.from_pandas(X_train)
val_data = Dataset.from_pandas(X_valid)


In [11]:
def evaluate(y_true, y_pred):

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels output: {0,1,2,3,4,5}

    # nested for loops
    for label in unique_labels:
        # will output a list of the index of one emotion at a time
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        # will output the list of one emotion
        label_y_true = [y_true[i] for i in label_indices]
        # label_y_true = [label for i in range(len(y_true))]
        # will output list of the predicted emotion in the same order as label_y_true
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2, 3, 4, 5, 6])
    print('\nConfusion Matrix:')
    print(conf_matrix)

# **Model**

In [13]:
notebook_login()

In [14]:
compute_dtype = getattr(torch, "float16")

def create_model_and_tokenizer():
  # defining a config for less powerful hardware to run the model
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=compute_dtype,
      bnb_4bit_use_double_quant=True,
  )

  model = AutoModelForCausalLM.from_pretrained(
      MODEL_NAME,
      use_safetensors=True,
      quantization_config=bnb_config,
      trust_remote_code=True,
      device_map="auto"
  )

  tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.padding_side = "right"

  return model, tokenizer

In [15]:
model, tokenizer = create_model_and_tokenizer()
#model.config.use_cache = False
#model.resize_token_embeddings(32000)
model.config.pretraining_tp = 1
model, tokenizer = setup_chat_format(model, tokenizer)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

c:\Users\alice\Documents\Final year project\Fyp_code\.venv\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alice\.cache\huggingface\hub\models--meta-llama--Llama-2-7b-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [16]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 '_load_in_8bit': False,
 '_load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': True,
 'bnb_4bit_compute_dtype': 'float16',
 'load_in_4bit': True,
 'load_in_8bit': False}

In [17]:
def predict(model, tokenizer):
    y_pred = []
    none_pred = []
    for i in tqdm(range(len(X_test))):
        # prompt = X_test.iloc[i]["text"].to(DEVICE)
        prompt = X_test.iloc[i]["text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 4,
                        temperature = 0.1, # lower temp = less random, higher temp = more random or unexpected text
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        #print(result)
        #print(answer)
        if "sadness" in answer:
            y_pred.append(0)
        elif "joy" in answer:
            y_pred.append(1)
        elif "love" in answer:
            y_pred.append(2)
        elif "anger" in answer:
            y_pred.append(3)
        elif "fear" in answer:
            y_pred.append(4)
        elif "surprise" in answer:
            y_pred.append(5)
        else:
            y_pred.append(6)
            none_pred.append(answer)
        # print(none_pred)
    return y_pred, none_pred

In [18]:
X_test.iloc[1]["text"]

"Analyze the emotion of the texts in the square brackets,\n            determine if it is sadness, joy, love, anger, fear, surprise, and return the answer as\n            the corresponding sentiment label 'sadness','joy','love','anger', 'fear', 'surprise'.\n\n            [im updating my blog because i feel shitty] ="

In [24]:
y_pred, none_pred = predict(model, tokenizer)

100%|██████████| 2000/2000 [05:03<00:00,  6.59it/s]


In [25]:
# none_pred
evaluate(y_true, y_pred)

Accuracy: 0.496
Accuracy for label 0: 0.702
Accuracy for label 1: 0.501
Accuracy for label 2: 0.434
Accuracy for label 3: 0.513
Accuracy for label 4: 0.076
Accuracy for label 5: 0.152

Classification Report:
              precision    recall  f1-score   support

           0       0.47      0.70      0.56       581
           1       0.75      0.50      0.60       695
           2       0.27      0.43      0.34       159
           3       0.44      0.51      0.47       275
           4       0.81      0.08      0.14       224
           5       0.28      0.15      0.20        66
           6       0.00      0.00      0.00         0

    accuracy                           0.50      2000
   macro avg       0.43      0.34      0.33      2000
weighted avg       0.58      0.50      0.49      2000


Confusion Matrix:
[[408  46  37  80   1   2   7]
 [174 348 109  38   3   5  18]
 [ 52  27  69   9   0   0   2]
 [ 96  14  17 141   0   2   5]
 [114  14  11  47  17  17   4]
 [ 24  16   8   6   0

/home/liamo/Vscode/FYP_Code/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/liamo/Vscode/FYP_Code/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/liamo/Vscode/FYP_Code/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
lora_alpha = 32
lora_dropout = 0.05
lora_r = 16

peft_config = LoraConfig(
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    r = lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [27]:
print(model.init_weights())

None


# **Training**

In [20]:
output_dir = "./output"

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=3,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,                         # log every 10 steps
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=True,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="tensorboard",                  # report metrics to tensorboard
    evaluation_strategy="epoch"               # save checkpoint every epoch
)
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=val_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=1024,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

c:\Users\alice\Documents\Final year project\Fyp_code\.venv\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [21]:
# Train model
trainer.train()

# Save trained model and tokenizer
trainer.save_model()
tokenizer.save_pretrained(output_dir)

  0%|          | 0/6000 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
c:\Users\alice\Documents\Final year project\Fyp_code\.venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:665: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


KeyboardInterrupt: 

In [32]:
%load_ext tensorboard
%tensorboard --logdir logs/runs

ModuleNotFoundError: No module named 'tensorboard'

In [16]:
model = AutoModelForCausalLM.from_pretrained("./Trained_Model")
tokenizer = AutoTokenizer.from_pretrained("./Trained_Model")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
y_pred = predict(model, tokenizer)
evaluate(y_true, y_pred)